<a href="https://colab.research.google.com/github/twisha-k/Python_notes/blob/main/41_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 41: Air Quality Analysis - Grouping & Aggregation

### Teacher-Student Activity

In this class, we will learn grouping and aggregation. These are the most frequently used operations on a DataFrame as far as analysis is concerned.

First, we will group the DataFrame by a particular set of values and then we will apply the aggregation operations such as count, sum, median, mean, standard deviation etc.

Let's run the codes we covered in the previous classes and continue this session from **Activity 1: Univariate Grouping** section.

---

### Recap

Run the code cell below to load the dataset and apply the operations on the DataFrame that were covered in the previous classes.

In [ ]:
# Run the code cell.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


# Loading the dataset.
csv_file = 'https://s3-student-datasets-bucket.whjr.online/whitehat-ds-datasets/air-quality/AirQualityUCI.csv'
df = pd.read_csv(csv_file, sep=';')

# Dropping the 'Unnamed: 15' & 'Unnamed: 16' columns.
df = df.drop(columns=['Unnamed: 15', 'Unnamed: 16'], axis=1)

# Dropping the null values.
df = df.dropna()

# Creating a Pandas series containing 'datetime' objects.
dt_series = pd.Series(data = [item.split("/")[2] + "-" + item.split("/")[1] + "-" + item.split("/")[0] for item in df['Date']], index=df.index) + ' ' + pd.Series(data=[str(item).replace(".", ":") for item in df['Time']], index=df.index)
dt_series = pd.to_datetime(dt_series)

# Remove the Date & Time columns from the DataFrame and insert the 'dt_series' in it.
df = df.drop(columns=['Date', 'Time'], axis=1)
df.insert(loc=0, column='DateTime', value=dt_series)

# Get the Pandas series containing the year values as integers.
year_series = dt_series.dt.year

# Get the Pandas series containing the month values as integers.
month_series = dt_series.dt.month

# Get the Pandas series containing the day values as integers.
day_series = dt_series.dt.day

# Get the Pandas series containing the days of a week, i.e., Monday, Tuesday, Wednesday etc.
day_name_series = dt_series.dt.day_name()

# Add the 'Year', 'Month', 'Day' and 'Day Name' columns to the DataFrame.
df['Year'] = year_series
df['Month'] = month_series
df['Day'] = day_series
df['Day Name'] = day_name_series

# Sort the DataFrame by the 'DateTime' values in the ascending order. Also, display the first 10 rows of the DataFrame.
df = df.sort_values(by='DateTime')

# Create a function to replace the commas with periods in a Pandas series.
def comma_to_period(series):
    new_series = pd.Series(data=[float(str(item).replace(',', '.')) for item in series], index=df.index)
    return new_series

# Apply the 'comma_to_period()' function on the ''CO(GT)', 'C6H6(GT)', 'T', 'RH' and 'AH' columns.
cols_to_correct = ['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH'] # Create a list of column names.
for col in cols_to_correct: # Iterate through each column
    df[col] = comma_to_period(df[col]) # Replace the original column with the new series.

# Remove all the columns from the 'df' DataFrame containing more than 10% garbage value.
df = df.drop(columns=['NMHC(GT)', 'CO(GT)', 'NOx(GT)', 'NO2(GT)'], axis=1)

# Create a new DataFrame containing records for the years 2004 and 2005.
aq_2004_df = df[df['Year'] == 2004]
aq_2005_df = df[df['Year'] == 2005]

# Replace the -200 value with the median values for each column having indices between 1 and -4 (excluding -4) for the 2004 year DataFrame.
for col in aq_2004_df.columns[1:-4]:
  median = aq_2004_df.loc[aq_2004_df[col] != -200, col].median() # Get the median value for each column after excluding -200.
  aq_2004_df[col] = aq_2004_df[col].replace(to_replace=-200, value=median)

# Repeat the same exercise for the 2005 year DataFrame.
for col in aq_2005_df.columns[1:-4]:
  median = aq_2005_df.loc[aq_2005_df[col] != -200, col].median()
  aq_2005_df[col] = aq_2005_df[col].replace(to_replace=-200, value=median)

---

#### Activity 1: Univariate Grouping

Before we begin grouping and aggregation, let's first get the list of columns we are left with after applying all the previous operations.

In [ ]:
# S1.1: Get the list of columns present in the DataFrame.


The description for all the columns containing data for air pollutants, temperature, relative humidity and absolute humidity is provided below.


|Columns|Description|
|-|-|
|PT08.S1(CO)|PT08.S1 (tin oxide) hourly averaged sensor response (nominally $\text{CO}$ targeted)|
|C6H6(GT)|True hourly averaged Benzene concentration in $\frac{\mu g}{m^3}$|
|PT08.S2(NMHC)|PT08.S2 (titania) hourly averaged sensor response (nominally $\text{NMHC}$ targeted)|
|PT08.S3(NOx)|PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_x$ targeted)|
|PT08.S4(NO2)|PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally $\text{NO}_2$ targeted)|
|PT08.S5(O3) |PT08.S5 (indium oxide) hourly averaged sensor response (nominally $\text{O}_3$ targeted)|
|T|Temperature in Â°C|
|RH|Relative Humidity (%)|
|AH|AH Absolute Humidity|

**The `groupby()` Function**

Now, let's group all the values by months, i.e., club all the values together for January, February and so on...

To group the values of a DataFrame by a particular value, use the `groupby()` function. Inside the function, you have to pass the column name to `by` parameter to specify the column about which you want to group the values.

**Syntax:** `dataframe.groupby(by='column')`

You can additionally set the `sort` parameter either equal to `True` or `False` depending on whether you want to sort the values or not. By default, the `groupby()` function returns a new Pandas `DataFrameGroupBy` object containing the grouped sorted values.

In [ ]:
# T1.1: Group the records for the 2004 DataFrame together by month.

group_2004=aq_2004_df.groupby(by='Month')

The `group_by()` function returns a `DataFrameGroupBy` object with its physical memory location.

How you want to group the values depends on the problem statement at hand. Here, we would like to analyse data month-wise. Hence, we have grouped the DataFrame by the `Month` column.

**The `get_group()` Function**

To get all the occurrences of a value from a group, use the `get_group()` function.

**Syntax:** `DataFrameGroupBy_object.get_group(item)`

E.g., if you want to get all the records for the month of March from the `group_2004_month DataFrameGroupBy` object, pass `3` as input to the `get_group()` function.

In [ ]:
# T1.2: Get all the records for the month of March.
#group_2004.groups
aq_2004_df.groupby(by='Month').groups

{3: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...], 4: [510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, ...], 5: [1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1

In [ ]:
aq_2004_df.groupby(by='Month').get_group(3)

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
0,2004-03-10 18:00:00,1360.0,11.9,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578,2004,3,10,Wednesday
1,2004-03-10 19:00:00,1292.0,9.4,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255,2004,3,10,Wednesday
2,2004-03-10 20:00:00,1402.0,9.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502,2004,3,10,Wednesday
3,2004-03-10 21:00:00,1376.0,9.2,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867,2004,3,10,Wednesday
4,2004-03-10 22:00:00,1272.0,6.5,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888,2004,3,10,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2004-03-31 19:00:00,1254.0,11.5,1033.0,819.0,1669.0,1068.0,14.3,52.9,0.8581,2004,3,31,Wednesday
506,2004-03-31 20:00:00,1198.0,9.3,953.0,882.0,1588.0,1019.0,13.2,56.0,0.8446,2004,3,31,Wednesday
507,2004-03-31 21:00:00,1060.0,5.5,787.0,1044.0,1418.0,833.0,12.6,57.0,0.8315,2004,3,31,Wednesday
508,2004-03-31 22:00:00,1050.0,5.1,769.0,1078.0,1409.0,759.0,12.3,57.8,0.8224,2004,3,31,Wednesday


The above DataFrame contains all the records only for March 2004.

In [ ]:
# S1.2: Get all the records for the month of April.
aq_2004_df.groupby(by='Month').get_group(4)

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
510,2004-04-01 00:00:00,1143.0,6.3,825.0,986.0,1477.0,978.0,12.0,61.6,0.8593,2004,4,1,Thursday
511,2004-04-01 01:00:00,1044.0,5.1,770.0,1031.0,1425.0,944.0,11.5,63.9,0.8652,2004,4,1,Thursday
512,2004-04-01 02:00:00,1034.0,4.1,716.0,1085.0,1405.0,891.0,10.7,67.2,0.8630,2004,4,1,Thursday
513,2004-04-01 03:00:00,956.0,4.0,713.0,1099.0,1422.0,849.0,9.0,73.1,0.8394,2004,4,1,Thursday
514,2004-04-01 04:00:00,909.0,2.4,615.0,1237.0,1322.0,790.0,10.2,66.6,0.8299,2004,4,1,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2004-04-30 19:00:00,1626.0,26.9,1476.0,555.0,2356.0,1786.0,19.8,58.6,1.3392,2004,4,30,Friday
1226,2004-04-30 20:00:00,1449.0,19.5,1282.0,625.0,2100.0,1569.0,19.1,61.1,1.3345,2004,4,30,Friday
1227,2004-04-30 21:00:00,1363.0,15.1,1152.0,684.0,1951.0,1495.0,18.2,65.4,1.3529,2004,4,30,Friday
1228,2004-04-30 22:00:00,1371.0,14.6,1136.0,689.0,1927.0,1471.0,18.1,66.1,1.3579,2004,4,30,Friday


The above DataFrame contains all the records only for April 2004.

Similarly, you can get values for all other months from the `grouped_2004_month DataFrameGroupBy` object.

Now, you group the records of 2005 about the `Month` column so that we can later compare the situation in 2004 with 2005.

In [ ]:
# S1.3: Group the records for the 2005 DataFrame together by month.
group_2005=aq_2005_df.groupby(by='Month')

---

#### Activity 2: Aggregation on Univariate Grouping^

You have already learnt the aggregation functions such as `count(), sum(), min(), max(), mean()` etc. Let's apply a few of them on the `DataFrameGroupBy` objects that we have already created.

The pollution is most severe during the winter season in any part of the world. The winter season in Italy begins from the last week of October and lasts till the last week of March. So, let's get the descriptive statistics for the concentrations of the air pollutants, temperature, relative humidity and absolute humidity in January, February, March, November and December in 2004 and 2005.

In [ ]:
# S2.1: Get the descriptive statistics for March 2004.
group_2004.get_group(3).describe()

,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day
count,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.0,510.0,510.000000
mean,1222.819608,9.932745,935.643137,1029.172549,1572.072549,1027.494118,14.390784,50.170784,0.789413,2004.0,3.0,20.870588
std,241.038541,7.060527,258.997406,268.130898,280.525357,381.600387,4.447539,14.265312,0.139932,0.0,0.0,6.141338
min,818.000000,0.600000,457.000000,537.000000,1050.000000,341.000000,6.100000,14.900000,0.402300,2004.0,3.0,10.000000
25%,1027.250000,4.600000,742.250000,834.250000,1359.000000,762.500000,11.000000,38.550000,0.691175,2004.0,3.0,16.000000
50%,1197.000000,8.650000,926.000000,966.500000,1543.000000,992.500000,14.050000,52.200000,0.792050,2004.0,3.0,21.000000
75%,1383.000000,13.650000,1106.250000,1175.500000,1730.000000,1272.000000,16.800000,60.750000,0.895900,2004.0,3.0,26.000000
max,2040.000000,39.200000,1754.000000,1935.000000,2679.000000,2359.000000,29.300000,83.200000,1.094500,2004.0,3.0,31.000000


In [ ]:
# S2.2: Get the descriptive statistics for March 2005.
group_2005.get_group(3).describe()

,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day
count,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.000000,744.0,744.0,744.000000
mean,1145.223118,8.726478,887.740591,709.447581,1225.188172,1083.598118,13.213306,50.909677,0.781531,2005.0,3.0,16.000000
std,197.140374,6.677222,253.315169,216.393926,275.936945,408.951492,6.394258,16.598389,0.291686,0.0,0.0,8.950289
min,715.000000,0.200000,387.000000,330.000000,551.000000,221.000000,-1.900000,13.500000,0.184700,2005.0,3.0,1.000000
25%,996.000000,3.700000,692.500000,551.750000,1054.750000,808.500000,8.675000,37.400000,0.591475,2005.0,3.0,8.000000
50%,1126.000000,6.600000,841.000000,684.500000,1217.000000,1039.500000,13.500000,51.800000,0.783400,2005.0,3.0,16.000000
75%,1267.500000,12.100000,1052.500000,830.000000,1376.000000,1365.250000,17.600000,64.575000,0.991450,2005.0,3.0,24.000000
max,1818.000000,35.500000,1675.000000,1804.000000,2147.000000,2159.000000,28.800000,84.000000,1.393000,2005.0,3.0,31.000000


If we keep on aggregating the values month-wise one-by-one, then it will become a cumbersome task. Let's get the descriptive statistics for all the months for both the years 2004 and 2005.

In [ ]:
# S2.3: Get the descriptive statistics for all the months for the year 2004.
group_2004.describe()

PT08.S1(CO)                                  ...   Day                  
            count         mean         std    min  ...   25%   50%   75%   max
Month                                              ...                        
3           510.0  1222.819608  241.038541  818.0  ...  16.0  21.0  26.0  31.0
4           720.0  1158.993056  227.826933  753.0  ...   8.0  15.5  23.0  30.0
5           744.0  1076.350806  193.766317  736.0  ...   8.0  16.0  24.0  31.0
6           720.0  1022.862500  157.922850  708.0  ...   8.0  15.5  23.0  30.0
7           744.0  1046.452957  158.314118  723.0  ...   8.0  16.0  24.0  31.0
8           744.0   986.259409  117.964334  711.0  ...   8.0  16.0  24.0  31.0
9           720.0  1081.022222  197.420922  718.0  ...   8.0  15.5  23.0  30.0
10          744.0  1184.745968  243.017236  704.0  ...   8.0  16.0  24.0  31.0
11          720.0  1132.148611  273.058383  647.0  ...   8.0  15.5  23.0  30.0
12          744.0  1084.196237  211.089768  691.0  ...   8.0  16.0  24.0  31.0

[10 rows x 88 columns]

In [ ]:
# S2.4: Get the descriptive statistics for all the months for the year 2005.
group_2005.describe()

PT08.S1(CO)                                  ...   Day                   
            count         mean         std    min  ...   25%   50%    75%   max
Month                                              ...                         
1           744.0  1108.086022  214.147270  744.0  ...  8.00  16.0  24.00  31.0
2           672.0  1086.369048  184.077312  755.0  ...  7.75  14.5  21.25  28.0
3           744.0  1145.223118  197.140374  715.0  ...  8.00  16.0  24.00  31.0
4            87.0   953.712644  125.441352  811.0  ...  1.00   2.0   3.00   4.0

[4 rows x 88 columns]

We still have quite large DataFrames containing the descriptive statistics for all the months and all the columns. Let's try to retrieve only the mean, standard deviation and median values for all the months.

---

#### Activity 3: The `agg()` Function^^^

Instead of applying the aggregation functions on a `DataFrameGroupBy` object individually, you can apply them together using the `agg()` function. Inside the `agg()` function, you have to pass the names of the functions (as string values) as inputs.

**Syntax:** `DataFrameGroupBy.agg(func=('func1', 'func2' ... 'funcN'))`

where `'func1', 'func2' ... 'funcN'` are the names of the aggregation functions to be applied. They are passed as a singular value (through tuple) to the `func` parameter.

Let's apply the `mean(), std()` and `median()` function on the `group_2004_month DataFrameGroupBy` object.

In [ ]:
# T3.1: Get mean, standard deviation and median for all the months for the year 2004.
group_2004.agg(func=('count','median','std'))

PT08.S1(CO)                     C6H6(GT)  ... Year   Day                 
            count  median         std    count  ...  std count median       std
Month                                           ...                            
3             510  1197.0  241.038541      510  ...  0.0   510   21.0  6.141338
4             720  1115.5  227.826933      720  ...  0.0   720   15.5  8.661458
5             744  1053.0  193.766317      744  ...  0.0   744   16.0  8.950289
6             720  1030.5  157.922850      720  ...  0.0   720   15.5  8.661458
7             744  1032.0  158.314118      744  ...  0.0   744   16.0  8.950289
8             744   987.5  117.964334      744  ...  0.0   744   16.0  8.950289
9             720  1060.0  197.420922      720  ...  0.0   720   15.5  8.661458
10            744  1159.5  243.017236      744  ...  0.0   744   16.0  8.950289
11            720  1106.5  273.058383      720  ...  0.0   720   15.5  8.661458
12            744  1060.0  211.089768      744  ...  0.0   744   16.0  8.950289

[10 rows x 33 columns]

You can also provide the name of the functions to be applied as items in a list.

In [ ]:
# S3.1: Get mean, standard deviation and median for all the months for the year 2005.
group_2005.agg(func=('mean','std','median'))

PT08.S1(CO)                      ...        Day                 
              mean         std  median  ...       mean       std median
Month                                   ...                            
1      1108.086022  214.147270  1076.0  ...  16.000000  8.950289   16.0
2      1086.369048  184.077312  1070.0  ...  14.500000  8.083764   14.5
3      1145.223118  197.140374  1126.0  ...  16.000000  8.950289   16.0
4       953.712644  125.441352   925.0  ...   2.344828  1.065610    2.0

[4 rows x 33 columns]

**Note:** You also choose to apply different aggregation functions on different columns by passing a dictionary containing the names of the columns and the aggregation function(s) to be applied on them.

E.g., let's apply the `mean(), std()` & `median()` functions on the `T` (temperature) column and `max()` function on the `AH` (Absolute humidity) column.

In [ ]:
# T3.2: On 'group_2004_month', apply the 'mean', 'std' & 'median' functions on the 'T' column and 'max' function on the 'AH' column.
# Also apply the 'count' function on the 'Month' column.
group_2004.agg(func={'T':['mean','std','median'],'AH':['max']})

T                       AH
            mean       std median     max
Month                                    
3      14.390784  4.447539  14.05  1.0945
4      16.935694  4.758767  16.30  1.4852
5      20.283468  5.163229  19.30  1.6296
6      26.155833  5.955815  24.70  1.9390
7      29.406989  6.178302  28.05  2.0042
8      28.555376  5.469489  27.55  2.1806
9      24.514861  5.440697  23.60  2.2310
10     20.497446  3.586663  20.30  2.0224
11     13.485417  4.990447  13.25  1.9800
12     12.136425  4.462879  12.00  1.3713

The least number of observations were recorded in March for the year 2004.

In [ ]:
# S3.2: On the 'group_2005_month', apply the 'min' & 'max' functions on the 'AH' and 'RH' columns.
# Also apply the 'count' function on the 'Month' column.
group_2005.agg(func={'T':['mean','std','median'],'AH':['max']})

T                       AH
            mean       std median     max
Month                                    
1       8.294624  3.701922   8.10  1.0567
2       7.338244  3.338817   7.35  1.0859
3      13.213306  6.394258  13.50  1.3930
4      16.660920  5.425207  15.10  0.8642

In [ ]:
group_2005.agg(func={'T':['mean','std','median'],'RH':['max']})

T                     RH
            mean       std median   max
Month                                  
1       8.294624  3.701922   8.10  86.6
2       7.338244  3.338817   7.35  86.6
3      13.213306  6.394258  13.50  84.0
4      16.660920  5.425207  15.10  63.1

In [ ]:
group_2005.agg(func={'T':['mean','std','median'],'AH':['min']})

T                       AH
            mean       std median     min
Month                                    
1       8.294624  3.701922   8.10  0.2477
2       7.338244  3.338817   7.35  0.2269
3      13.213306  6.394258  13.50  0.1847
4      16.660920  5.425207  15.10  0.3866

In [ ]:
group_2005.agg(func={'T':['mean','std','median'],'RH':['min']})

T                     RH
            mean       std median   min
Month                                  
1       8.294624  3.701922   8.10  17.2
2       7.338244  3.338817   7.35  18.0
3      13.213306  6.394258  13.50  13.5
4      16.660920  5.425207  15.10   9.9

The least number of observations were recorded in April for the year 2005.

---

#### Activity 4: Slicing `DataFrameGroupBy` Object^^


We still want to know the mean, standard deviation and median values only for the winter season for 2004 and 2005.

So, we can use the `loc[]` function to get the rows only for the winter season.

In [ ]:
# T4.1: Get mean, standard deviation and median values for the winter season of 2004.
group_2004.agg(func = ["std","median","mean"]).loc[[3,11,12],:]

PT08.S1(CO)                       ...       Day                  
              std  median         mean  ...       std median       mean
Month                                   ...                            
3      241.038541  1197.0  1222.819608  ...  6.141338   21.0  20.870588
11     273.058383  1106.5  1132.148611  ...  8.661458   15.5  15.500000
12     211.089768  1060.0  1084.196237  ...  8.950289   16.0  16.000000

[3 rows x 33 columns]

But we don't want any of the aggregated values for the `Year` and `Day` columns. Hence, we need to specify the column indices for the columns we want to be displayed.

In [ ]:
# T4.2: Get the column indices for the 'group_2004_month.agg(func=['mean', 'std', 'median'])' DataFrame columns.
group_2004.agg(func=['std','median','mean']).columns

MultiIndex([(  'PT08.S1(CO)',    'std'),
            (  'PT08.S1(CO)', 'median'),
            (  'PT08.S1(CO)',   'mean'),
            (     'C6H6(GT)',    'std'),
            (     'C6H6(GT)', 'median'),
            (     'C6H6(GT)',   'mean'),
            ('PT08.S2(NMHC)',    'std'),
            ('PT08.S2(NMHC)', 'median'),
            ('PT08.S2(NMHC)',   'mean'),
            ( 'PT08.S3(NOx)',    'std'),
            ( 'PT08.S3(NOx)', 'median'),
            ( 'PT08.S3(NOx)',   'mean'),
            ( 'PT08.S4(NO2)',    'std'),
            ( 'PT08.S4(NO2)', 'median'),
            ( 'PT08.S4(NO2)',   'mean'),
            (  'PT08.S5(O3)',    'std'),
            (  'PT08.S5(O3)', 'median'),
            (  'PT08.S5(O3)',   'mean'),
            (            'T',    'std'),
            (            'T', 'median'),
            (            'T',   'mean'),
            (           'RH',    'std'),
            (           'RH', 'median'),
            (           'RH',   'mean'),
            (   

Here, we have a collection of column names. For each column, the column itself and the subcolumns (`mean, std, median`) are put together in a tuple. So, for every column, we have three tuples.

We don't want the last six combinations of columns and sub columns to be part of our final aggregated DataFrame. Hence, we will pass the remaining column indices to the `loc[]` function.

In [ ]:
group_2004.agg(func = ["std","median","mean"]).columns[:-6]

MultiIndex([(  'PT08.S1(CO)',    'std'),
            (  'PT08.S1(CO)', 'median'),
            (  'PT08.S1(CO)',   'mean'),
            (     'C6H6(GT)',    'std'),
            (     'C6H6(GT)', 'median'),
            (     'C6H6(GT)',   'mean'),
            ('PT08.S2(NMHC)',    'std'),
            ('PT08.S2(NMHC)', 'median'),
            ('PT08.S2(NMHC)',   'mean'),
            ( 'PT08.S3(NOx)',    'std'),
            ( 'PT08.S3(NOx)', 'median'),
            ( 'PT08.S3(NOx)',   'mean'),
            ( 'PT08.S4(NO2)',    'std'),
            ( 'PT08.S4(NO2)', 'median'),
            ( 'PT08.S4(NO2)',   'mean'),
            (  'PT08.S5(O3)',    'std'),
            (  'PT08.S5(O3)', 'median'),
            (  'PT08.S5(O3)',   'mean'),
            (            'T',    'std'),
            (            'T', 'median'),
            (            'T',   'mean'),
            (           'RH',    'std'),
            (           'RH', 'median'),
            (           'RH',   'mean'),
            (   

In [ ]:
# T4.3: Get mean, standard deviation and median values for the winter season of 2004 without the 'Year' & 'Day' columns.
group_2004.agg(func = ["std","median","mean"]).loc[[3,11,12],group_2004.agg(func = ["std","median","mean"]).columns[:-6]]

PT08.S1(CO)                       ...        AH                   
              std  median         mean  ...       std   median      mean
Month                                   ...                             
3      241.038541  1197.0  1222.819608  ...  0.139932  0.79205  0.789413
11     273.058383  1106.5  1132.148611  ...  0.360286  0.95040  0.939991
12     211.089768  1060.0  1084.196237  ...  0.291196  0.85780  0.814761

[3 rows x 27 columns]

The `group_2004_month.agg(func=['mean', 'std', 'median']).columns[:-6]` is a list of combinations of all the columns and sub columns except for the last 6 combinations.

In [ ]:
# S4.1: Get mean, standard deviation and median values for the winter season of 2005 without the 'Year' & 'Day' columns.
#in 2004 grouped data-->months 3-12------>3,11,12 are winter seasons
#in 2005 grouped data-->months 1-4--->1,2,3 are winter seaon
group_2005.agg(func=['std','mean','median']).loc[[1,2,3],group_2005.agg(func=['std','mean','median']).columns[:-6]]

PT08.S1(CO)                       ...        AH                  
              std         mean  median  ...       std      mean  median
Month                                   ...                            
1      214.147270  1108.086022  1076.0  ...  0.218392  0.634713  0.6304
2      184.077312  1086.369048  1070.0  ...  0.144846  0.530231  0.5208
3      197.140374  1145.223118  1126.0  ...  0.291686  0.781531  0.7834

[3 rows x 27 columns]

**Note:** Unfortunately, the `quantile()` function cannot be applied as an aggregation function on a `DataFrameGroupBy` object. To get quartiles, use the `describe()` function.

Let's have a look on how we can get only quartiles using the `describe()` function.

In [ ]:
# T4.4: Get the list of all the columns of the 'aq_2004_df' DataFrame.
aq_2004_df.columns

Index(['DateTime', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)',
       'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH', 'Year', 'Month', 'Day',
       'Day Name'],
      dtype='object')

In [ ]:
aq_2004_df.columns[1:-4]

Index(['PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)',
       'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH'],
      dtype='object')

In [ ]:
group_2004.describe().loc[[3,11,12],:]

PT08.S1(CO)                                  ...   Day                  
            count         mean         std    min  ...   25%   50%   75%   max
Month                                              ...                        
3           510.0  1222.819608  241.038541  818.0  ...  16.0  21.0  26.0  31.0
11          720.0  1132.148611  273.058383  647.0  ...   8.0  15.5  23.0  30.0
12          744.0  1084.196237  211.089768  691.0  ...   8.0  16.0  24.0  31.0

[3 rows x 88 columns]

In [ ]:
# T4.5: For winters of 2004, get the quartiles of all the air pollutants, temperature, relative humidity and absolute humidity.
for i in aq_2004_df.columns[1:-4]:
 print("column name",i)
 print( group_2004.describe().loc[[3,11,12],[(i,"25%"),(i,"50%"),(i,"75%")]])


column name PT08.S1(CO)
      PT08.S1(CO)                
              25%     50%     75%
Month                            
3         1027.25  1197.0  1383.0
11         912.75  1106.5  1311.0
12         927.75  1060.0  1199.0
column name C6H6(GT)
      C6H6(GT)              
           25%    50%    75%
Month                       
3        4.600   8.65  13.65
11       5.000  10.10  17.60
12       4.375   8.60  12.50
column name PT08.S2(NMHC)
      PT08.S2(NMHC)                
                25%    50%      75%
Month                              
3            742.25  926.0  1106.25
11           762.50  981.0  1227.25
12           730.75  926.0  1067.00
column name PT08.S3(NOx)
      PT08.S3(NOx)               
               25%    50%     75%
Month                            
3           834.25  966.5  1175.5
11          558.75  722.5   923.0
12          671.75  823.0   981.5
column name PT08.S4(NO2)
      PT08.S4(NO2)                 
               25%     50%      75%
Month    

First, we have applied the `describe()` function which will return all the descriptive statistics for the `DataFrameGroupBy` object. Then using the `loc[]` function we have retrieved only the quartile values for the winter season of 2004.

In [ ]:
# S4.2: For winters of 2005, get the quartiles of all the air pollutants, temperature, relative humidity and absolute humidity.
#[1,2,3] inside loc function
for i in aq_2005_df.columns[1:-4]:
 print("column name",i)
 print( group_2005.describe().loc[[1,2,3],[(i,"25%"),(i,"50%"),(i,"75%")]])


column name PT08.S1(CO)
      PT08.S1(CO)                 
              25%     50%      75%
Month                             
1          933.50  1076.0  1256.00
2          958.75  1070.0  1155.25
3          996.00  1126.0  1267.50
column name C6H6(GT)
      C6H6(GT)           
           25%  50%   75%
Month                    
1          3.6  6.3  13.1
2          3.6  6.3   9.5
3          3.7  6.6  12.1
column name PT08.S2(NMHC)
      PT08.S2(NMHC)                
                25%    50%      75%
Month                              
1            685.75  826.0  1087.25
2            688.75  826.0   960.00
3            692.50  841.0  1052.50
column name PT08.S3(NOx)
      PT08.S3(NOx)               
               25%    50%     75%
Month                            
1           590.75  740.0  932.00
2           656.75  740.0  897.25
3           551.75  684.5  830.00
column name PT08.S4(NO2)
      PT08.S4(NO2)                 
               25%     50%      75%
Month                

**Note:** Since we have grouped the DataFrames for the years 2004 and 2005 by the `Month` column, it becomes the default index column in the aggregated DataFrames. If you don't want it to happen, pass the `as_index=False` parameter inside the `groupby()` function.

In [ ]:
# S4.3: Group the DataFrame for the year 2004 by the 'Month' column without making it a default index column in the aggregated DataFrames.
# Apply the 'mean' function on the 'DataFrameGroupBy' object.
aq_2004_df.groupby(by='Month').describe()#month column wil act as index val when applies soe aggregation functions
aq_2004_df.groupby(by='Month',as_index = False).describe() #here month will not be an index column

PT08.S1(CO)                                  ...   Day                  
        count         mean         std    min  ...   25%   50%   75%   max
0       510.0  1222.819608  241.038541  818.0  ...  16.0  21.0  26.0  31.0
1       720.0  1158.993056  227.826933  753.0  ...   8.0  15.5  23.0  30.0
2       744.0  1076.350806  193.766317  736.0  ...   8.0  16.0  24.0  31.0
3       720.0  1022.862500  157.922850  708.0  ...   8.0  15.5  23.0  30.0
4       744.0  1046.452957  158.314118  723.0  ...   8.0  16.0  24.0  31.0
5       744.0   986.259409  117.964334  711.0  ...   8.0  16.0  24.0  31.0
6       720.0  1081.022222  197.420922  718.0  ...   8.0  15.5  23.0  30.0
7       744.0  1184.745968  243.017236  704.0  ...   8.0  16.0  24.0  31.0
8       720.0  1132.148611  273.058383  647.0  ...   8.0  15.5  23.0  30.0
9       744.0  1084.196237  211.089768  691.0  ...   8.0  16.0  24.0  31.0

[10 rows x 96 columns]

As you can see, the `Month` column is not an index column of the mean aggregated DataFrame.

---

#### Activity 5: Multivariate Grouping & Aggregation

We can also group multiple columns at once by passing a list of columns to be grouped.

Let's group the DataFrames about the `Month` & `Day Name` columns.

In [ ]:
# S5.1: Group the DataFrame for the year 2004 about the 'Month' & 'Day Name' columns.
group_month_day_2004=aq_2004_df.groupby(by = ['Month' , 'Day Name'])

In [ ]:
aq_2004_df.groupby(by = ['Month' , 'Day Name']).mean()

PT08.S1(CO)   C6H6(GT)  PT08.S2(NMHC)  ...        AH    Year   Day
Month Day Name                                          ...                        
3     Friday     1223.555556  11.358333     986.263889  ...  0.754518  2004.0  19.0
      Monday     1191.166667  10.358333     937.555556  ...  0.787414  2004.0  22.0
      Saturday   1238.138889   9.144444     925.972222  ...  0.767614  2004.0  20.0
      Sunday     1224.513889   7.600000     866.000000  ...  0.839960  2004.0  21.0
      Thursday   1227.263889   9.956944     928.083333  ...  0.794621  2004.0  18.0
...                      ...        ...            ...  ...       ...     ...   ...
12    Saturday   1134.541667   9.560417     945.031250  ...  0.781326  2004.0  14.5
      Sunday     1066.854167   7.417708     843.531250  ...  0.813534  2004.0  15.5
      Thursday   1080.116667  10.197500     930.558333  ...  0.821778  2004.0  16.0
      Tuesday    1085.125000  10.567708     959.062500  ...  0.779568  2004.0  17.5
      Wednesday  1046.358333   8.189167     878.675000  ...  0.854508  2004.0  15.0

[70 rows x 11 columns]

Let's get the count of each day for each month in 2004.


In [ ]:
# S5.2 Get the count of each day for each month in 2004.
aq_2005_df.groupby(by = ['Month' , 'Day Name']).mean()

PT08.S1(CO)   C6H6(GT)  PT08.S2(NMHC)  ...        AH    Year   Day
Month Day Name                                          ...                        
1     Friday     1192.781250  11.634375     990.750000  ...  0.678903  2005.0  17.5
      Monday     1105.708333   9.585833     913.841667  ...  0.585532  2005.0  17.0
      Saturday   1134.283333   8.990833     905.816667  ...  0.607258  2005.0  15.0
      Sunday     1014.850000   5.164167     749.608333  ...  0.569330  2005.0  16.0
      Thursday   1082.541667   8.415625     876.479167  ...  0.665009  2005.0  16.5
      Tuesday    1126.020833   9.372917     897.166667  ...  0.653023  2005.0  14.5
      Wednesday  1117.770833   9.110417     893.333333  ...  0.719436  2005.0  15.5
2     Friday     1100.406250   8.045833     879.000000  ...  0.557341  2005.0  14.5
      Monday      985.072917   5.678125     757.458333  ...  0.467448  2005.0  17.5
      Saturday   1141.708333   8.039583     872.718750  ...  0.515513  2005.0  15.5
      Sunday     1065.697917   5.209375     750.812500  ...  0.567748  2005.0  16.5
      Thursday   1130.447917   8.423958     886.645833  ...  0.577657  2005.0  13.5
      Tuesday    1076.093750   7.680208     843.875000  ...  0.494877  2005.0  11.5
      Wednesday  1105.156250   8.478125     892.302083  ...  0.531036  2005.0  12.5
3     Friday     1238.833333  11.610417    1000.843750  ...  0.794769  2005.0  14.5
      Monday     1124.291667   7.968750     855.291667  ...  0.848991  2005.0  17.5
      Saturday   1142.656250   6.917708     840.416667  ...  0.868167  2005.0  15.5
      Sunday     1074.645833   4.781250     732.145833  ...  0.876355  2005.0  16.5
      Thursday   1178.150000   9.501667     920.783333  ...  0.752872  2005.0  17.0
      Tuesday    1096.466667   9.030000     886.933333  ...  0.690262  2005.0  15.0
      Wednesday  1161.425000  10.550000     953.316667  ...  0.691732  2005.0  16.0
4     Friday      903.291667   3.412500     663.000000  ...  0.636529  2005.0   1.0
      Monday     1090.533333   8.440000     862.266667  ...  0.724787  2005.0   4.0
      Saturday    890.958333   2.529167     616.291667  ...  0.546167  2005.0   2.0
      Sunday      981.375000   4.316667     714.708333  ...  0.617583  2005.0   3.0

[25 rows x 11 columns]

Let's get all the records for all Mondays in the month of March, 2004.

**Note:** The values to be retrieved should be passed as a tuple in the same order of grouping.

In [ ]:
# S5.3: Get all the records for all Mondays in the month of March, 2004.
group_month_day_2004.get_group((3,'Monday'))

,DateTime,PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Year,Month,Day,Day Name
102,2004-03-15 00:00:00,1239.0,7.4,872.0,985.0,1547.0,1250.0,14.8,54.7,0.9164,2004,3,15,Monday
103,2004-03-15 01:00:00,1239.0,6.9,853.0,1010.0,1543.0,1174.0,14.0,57.0,0.9094,2004,3,15,Monday
104,2004-03-15 02:00:00,1224.0,7.0,855.0,998.0,1566.0,1149.0,13.4,61.3,0.9361,2004,3,15,Monday
105,2004-03-15 03:00:00,1078.0,4.4,734.0,1128.0,1487.0,1021.0,12.6,63.5,0.9230,2004,3,15,Monday
106,2004-03-15 04:00:00,1078.0,4.0,711.0,1150.0,1468.0,1013.0,12.3,65.4,0.9351,2004,3,15,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,2004-03-29 19:00:00,1127.0,8.2,907.0,957.0,1404.0,796.0,14.5,35.4,0.5839,2004,3,29,Monday
458,2004-03-29 20:00:00,1020.0,5.7,797.0,1066.0,1302.0,689.0,13.2,37.9,0.5733,2004,3,29,Monday
459,2004-03-29 21:00:00,949.0,3.8,700.0,1221.0,1178.0,553.0,13.1,34.4,0.5170,2004,3,29,Monday
460,2004-03-29 22:00:00,930.0,4.0,710.0,1278.0,1137.0,509.0,13.5,28.5,0.4405,2004,3,29,Monday


Similarly, you can retrieve records for different days in different months in a year.

Let's pause here. In the next class, we will continue with grouping, aggregation and learn to create time series plots. We will also learn how to customise the `matplotlib` plots.

**Note:** You may now attempt **Project 11**.

---

### Activities

1. Air Quality Analysis - Grouping & Aggregation (Class Copy):
https://colab.research.google.com/drive/1-dLZe4Y_esRwUirbGMiDCYBcPDgzVU3


---